In [16]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

### Load Vocab

In [1]:
vocab = {}
idToWord = {}
with open('vocab.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' : ')
        vocab[word] = index
        idToWord[int(index)] = word

### Create input squences

In [12]:
look_back_len = 5 + 1
sequences = []
vocabulary_size = len(vocab)

with open('int-seq.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        #print(numbers)
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    #print(sequences)

    n_sequences = np.empty([len(sequences), look_back_len], dtype='int32')
    for i in range(len(sequences)):
        n_sequences[i] = sequences[i]

n_sequences

array([[ 2, 13, 17,  9,  0,  2],
       [13, 17,  9,  0,  2, 18],
       [17,  9,  0,  2, 18,  7],
       [ 9,  0,  2, 18,  7,  1],
       [ 0,  2, 18,  7,  1,  4],
       [ 2, 18,  7,  1,  4, 19],
       [18,  7,  1,  4, 19,  8],
       [ 7,  1,  4, 19,  8, 20],
       [ 1,  4, 19,  8, 20, 21],
       [ 4, 19,  8, 20, 21,  0],
       [19,  8, 20, 21,  0, 22],
       [ 8, 20, 21,  0, 22,  3],
       [20, 21,  0, 22,  3,  2],
       [21,  0, 22,  3,  2,  1],
       [ 0, 22,  3,  2,  1,  4],
       [22,  3,  2,  1,  4,  9],
       [ 3,  2,  1,  4,  9,  0],
       [ 2,  1,  4,  9,  0,  0],
       [ 1,  4,  9,  0,  0,  2],
       [ 4,  9,  0,  0,  2, 23],
       [ 9,  0,  0,  2, 23,  8],
       [ 0,  0,  2, 23,  8,  1],
       [ 0,  2, 23,  8,  1, 10],
       [ 2, 23,  8,  1, 10, 14],
       [23,  8,  1, 10, 14,  1],
       [ 8,  1, 10, 14,  1,  4],
       [ 1, 10, 14,  1,  4,  5],
       [10, 14,  1,  4,  5,  0],
       [14,  1,  4,  5,  0,  2],
       [ 1,  4,  5,  0,  2,  3],
       [ 4

### Convert output to one hot encoded vector

In [15]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_targets[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Train Model

In [17]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 5)              185       
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 50)             11200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 37)                1887      
Total params: 36,022
Trainable params: 36,022
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
111/111 [==============================] - 2s 18ms/step - loss: 3.6088 - accuracy: 0.0901
Epoch 2/500
111/111 [==============================] - 0s 198us/step - loss: 3.5987 - accuracy: 0.1532
Epoch 3/500
111/111 [==============================] - 0s 180us/step - loss: 3.5860 - accuracy: 0.1532
Epoch 4/500
111/111 [==============================] - 0s 189us/step - loss: 3.5665 - accuracy: 0.1532
Epoch 5/500
111/111 [==============================] - 0s 207us/step - loss: 3.5390 - accuracy: 0.1532
Epoch 6/500
111/111 [==============================] - 0s 207us/step - loss: 3.4926 - accuracy: 0.1532
Epoch 7/500
111/111 [==============================] - 0s 189us/step - loss: 3.4221 - accuracy: 0.1532
Epoch 8/500
111/111 [==============================] - 0s 171us/step - loss: 3.3154 - accuracy: 0.1532
Epoch 9/500
111/111 [==============================] - 0s 171us/step - loss: 3.1816 - accuracy: 0.1532
Epoch 10/500
111/111 [==============================] - 0s 198us/step - lo

111/111 [==============================] - 0s 189us/step - loss: 2.4740 - accuracy: 0.2252
Epoch 80/500
111/111 [==============================] - 0s 189us/step - loss: 2.4568 - accuracy: 0.2162
Epoch 81/500
111/111 [==============================] - 0s 162us/step - loss: 2.4450 - accuracy: 0.2162
Epoch 82/500
111/111 [==============================] - 0s 189us/step - loss: 2.4294 - accuracy: 0.2162
Epoch 83/500
111/111 [==============================] - 0s 171us/step - loss: 2.4140 - accuracy: 0.2342
Epoch 84/500
111/111 [==============================] - 0s 180us/step - loss: 2.4027 - accuracy: 0.2342
Epoch 85/500
111/111 [==============================] - 0s 171us/step - loss: 2.3790 - accuracy: 0.2342
Epoch 86/500
111/111 [==============================] - 0s 198us/step - loss: 2.3672 - accuracy: 0.2342
Epoch 87/500
111/111 [==============================] - 0s 198us/step - loss: 2.3540 - accuracy: 0.2342
Epoch 88/500
111/111 [==============================] - 0s 180us/step - loss:

111/111 [==============================] - 0s 171us/step - loss: 1.7629 - accuracy: 0.3604
Epoch 158/500
111/111 [==============================] - 0s 189us/step - loss: 1.7448 - accuracy: 0.3694
Epoch 159/500
111/111 [==============================] - 0s 189us/step - loss: 1.7472 - accuracy: 0.3874
Epoch 160/500
111/111 [==============================] - 0s 189us/step - loss: 1.7395 - accuracy: 0.3784
Epoch 161/500
111/111 [==============================] - 0s 180us/step - loss: 1.7325 - accuracy: 0.3784
Epoch 162/500
111/111 [==============================] - 0s 189us/step - loss: 1.7253 - accuracy: 0.3784
Epoch 163/500
111/111 [==============================] - 0s 207us/step - loss: 1.7199 - accuracy: 0.3784
Epoch 164/500
111/111 [==============================] - 0s 180us/step - loss: 1.7134 - accuracy: 0.3874
Epoch 165/500
111/111 [==============================] - 0s 198us/step - loss: 1.7044 - accuracy: 0.4054
Epoch 166/500
111/111 [==============================] - 0s 189us/ste

111/111 [==============================] - 0s 189us/step - loss: 1.2898 - accuracy: 0.5495
Epoch 236/500
111/111 [==============================] - 0s 171us/step - loss: 1.2645 - accuracy: 0.5766
Epoch 237/500
111/111 [==============================] - 0s 162us/step - loss: 1.2775 - accuracy: 0.6216
Epoch 238/500
111/111 [==============================] - 0s 198us/step - loss: 1.2583 - accuracy: 0.6036
Epoch 239/500
111/111 [==============================] - 0s 198us/step - loss: 1.2396 - accuracy: 0.6396
Epoch 240/500
111/111 [==============================] - 0s 171us/step - loss: 1.2221 - accuracy: 0.6577
Epoch 241/500
111/111 [==============================] - 0s 207us/step - loss: 1.2304 - accuracy: 0.6126
Epoch 242/500
111/111 [==============================] - 0s 188us/step - loss: 1.2192 - accuracy: 0.6126
Epoch 243/500
111/111 [==============================] - 0s 189us/step - loss: 1.2181 - accuracy: 0.6216
Epoch 244/500
111/111 [==============================] - 0s 180us/ste

111/111 [==============================] - 0s 171us/step - loss: 0.9110 - accuracy: 0.7117
Epoch 314/500
111/111 [==============================] - 0s 216us/step - loss: 0.8995 - accuracy: 0.7207
Epoch 315/500
111/111 [==============================] - 0s 198us/step - loss: 0.8845 - accuracy: 0.7387
Epoch 316/500
111/111 [==============================] - 0s 207us/step - loss: 0.8833 - accuracy: 0.7207
Epoch 317/500
111/111 [==============================] - 0s 162us/step - loss: 0.8835 - accuracy: 0.7568
Epoch 318/500
111/111 [==============================] - 0s 189us/step - loss: 0.8796 - accuracy: 0.7477
Epoch 319/500
111/111 [==============================] - 0s 189us/step - loss: 0.8611 - accuracy: 0.7387
Epoch 320/500
111/111 [==============================] - 0s 171us/step - loss: 0.8668 - accuracy: 0.7117
Epoch 321/500
111/111 [==============================] - 0s 189us/step - loss: 0.8492 - accuracy: 0.7568
Epoch 322/500
111/111 [==============================] - 0s 216us/ste

111/111 [==============================] - 0s 207us/step - loss: 0.6140 - accuracy: 0.8288
Epoch 392/500
111/111 [==============================] - 0s 189us/step - loss: 0.6191 - accuracy: 0.8288
Epoch 393/500
111/111 [==============================] - 0s 189us/step - loss: 0.6158 - accuracy: 0.8288
Epoch 394/500
111/111 [==============================] - 0s 198us/step - loss: 0.6181 - accuracy: 0.8198
Epoch 395/500
111/111 [==============================] - 0s 180us/step - loss: 0.6152 - accuracy: 0.8378
Epoch 396/500
111/111 [==============================] - 0s 180us/step - loss: 0.6306 - accuracy: 0.8198
Epoch 397/500
111/111 [==============================] - 0s 171us/step - loss: 0.6285 - accuracy: 0.8198
Epoch 398/500
111/111 [==============================] - 0s 189us/step - loss: 0.6206 - accuracy: 0.8108
Epoch 399/500
111/111 [==============================] - 0s 180us/step - loss: 0.5953 - accuracy: 0.8288
Epoch 400/500
111/111 [==============================] - 0s 171us/ste

Epoch 468/500
111/111 [==============================] - 0s 198us/step - loss: 0.4060 - accuracy: 0.8649
Epoch 469/500
111/111 [==============================] - 0s 198us/step - loss: 0.4092 - accuracy: 0.8919
Epoch 470/500
111/111 [==============================] - 0s 207us/step - loss: 0.4035 - accuracy: 0.8919
Epoch 471/500
111/111 [==============================] - 0s 189us/step - loss: 0.4015 - accuracy: 0.9009
Epoch 472/500
111/111 [==============================] - 0s 207us/step - loss: 0.3953 - accuracy: 0.9009
Epoch 473/500
111/111 [==============================] - 0s 225us/step - loss: 0.3884 - accuracy: 0.9009
Epoch 474/500
111/111 [==============================] - 0s 207us/step - loss: 0.3881 - accuracy: 0.8919
Epoch 475/500
111/111 [==============================] - 0s 171us/step - loss: 0.3874 - accuracy: 0.9009
Epoch 476/500
111/111 [==============================] - 0s 180us/step - loss: 0.3834 - accuracy: 0.9099
Epoch 477/500
111/111 [==============================] 

### Test Model

In [19]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)

print("Top 3 Suggestions:")
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    print(idToWord[i])

[2, 13, 35, 5] [[ 0  2 13 35  5]]
Top 3 Suggestions:
:
n
(
